## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [56]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import statsmodels.stats.proportion as propstest

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044331,-0.385883,1,1,2,2


In [2]:
train_data.dtypes

ID             int64
Promotion     object
purchase       int64
V1             int64
V2           float64
V3           float64
V4             int64
V5             int64
V6             int64
V7             int64
dtype: object

In [3]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need

### 1. Determine how many more customers purchased product X

Data Exploration

In [16]:
#Looking at the number of observations
num_obs = train_data['Promotion'].shape[0]
num_obs

84534

In [17]:
#Reviewing the data in more detail
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84534 entries, 0 to 84533
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         84534 non-null  int64  
 1   Promotion  84534 non-null  object 
 2   purchase   84534 non-null  int64  
 3   V1         84534 non-null  int64  
 4   V2         84534 non-null  float64
 5   V3         84534 non-null  float64
 6   V4         84534 non-null  int64  
 7   V5         84534 non-null  int64  
 8   V6         84534 non-null  int64  
 9   V7         84534 non-null  int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 6.4+ MB


In [20]:
#number of people who bought and are in treatment group 
num_b_treat = train_data.loc[train_data['Promotion']== 'Yes','purchase'].sum()

#number of people that bought who are in the control groups
num_b_control = train_data.loc[train_data['Promotion'] == 'No','purchase'].sum()

print ('The number of people who bought from the treatment group is %d' %num_b_treat)
print ('The number of people who bought from the control group is %d' %num_b_control)

The number of people who bought from the treatment group is 721
The number of people who bought from the control group is 319


In [21]:
#number of people in treatment group
num_treat = train_data.loc[train_data['Promotion'] == 'Yes'].shape[0]
#number of people in control group
num_control = train_data.loc[train_data['Promotion'] == 'No'].shape[0]

print ('The number of people in the treatment group is %d' %num_treat)
print ('The number of people in the control group is %d' %num_control)


The number of people in the treatment group is 42364
The number of people in the control group is 42170


In [36]:
#calculating IRR

irr = (num_b_treat/num_treat) - (num_b_control/num_control)
print ('The incremental response rate is:%f' %irr)


The incremental response rate is:0.009455


In [23]:
#calculating NIR
nir = (10*num_b_treat - 0.15*num_treat) - 10*num_b_control
print ('The net incremental revenue is %d' %nir)

The net incremental revenue is -2334


### Modeling

We will try to predict which customers will buy the product based on their features v1-v7.

The process will go as follows:
- Randomly extract 1000 customers from people who didn't purchase the product, and combine them with customers who bought the product, so that the two group are of the same size.
- Convert categorical variables into dummy variables
- Split the sample into train group and test group, with test_size = 0.3
- Use the train group to train the Logistic Regression model
- Use the model to predict the purchasing behavior of the test group
- Evaluate the model

In [46]:
# Randomly select 1000 rows from not purchased dataset
# Combine it with all the rows from purchased dataset
pur = train_data[train_data['purchase'] == 1]
not_pur = train_data[train_data['purchase'] == 0].sample(1000)
sample = pd.concat([pur, not_pur], axis = 0)
sample.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
107,162,Yes,1,1,33.030562,-0.732194,2,1,1,1
328,503,Yes,1,3,31.760033,1.518828,2,1,2,2
352,534,Yes,1,2,17.512709,1.259095,2,3,4,2
383,580,Yes,1,1,31.043736,-1.338239,2,1,2,1
423,639,No,1,1,34.901993,-0.905350,2,3,2,2


In [50]:
# Converting categorical variables into dummy variables
cat_var = train_data.select_dtypes('int').columns
cat_var = cat_var.drop(['ID', 'purchase'])

features_df = pd.get_dummies(data = sample, columns = cat_var)
features_df.head()


,ID,Promotion,purchase,V2,V3,V1_0,V1_1,V1_2,V1_3,V4_1,...,V5_1,V5_2,V5_3,V5_4,V6_1,V6_2,V6_3,V6_4,V7_1,V7_2
107,162,Yes,1,33.030562,-0.732194,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
328,503,Yes,1,31.760033,1.518828,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
352,534,Yes,1,17.512709,1.259095,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
383,580,Yes,1,31.043736,-1.338239,0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,0
423,639,No,1,34.901993,-0.905350,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [61]:
# Split them into train data and test data
x = features_df.drop(['ID','Promotion','purchase'], axis =1)
y = features_df['purchase']

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=10)

In [62]:
# Model training with logistic regression
lgr = LogisticRegression()

lgr.fit(train_x, train_y)

LogisticRegression()

In [63]:
# Testing the model with test_x to predict pred_y
pred_y = lgr.predict(test_x)

# Compute the precision rate and recall rate
precision = precision_score(test_y, pred_y)
recall = recall_score(test_y, pred_y)

print('The precision score of the model is %.4f' %precision)
print('The recall score of the model is %.4f' %recall)

The precision score of the model is 0.6181
The recall score of the model is 0.5859


In [64]:
# Use a confusion matrix to show the comparison of the predicted promotion and the actual promotion
con_mat = pd.DataFrame(confusion_matrix(pred_y, test_y), index = ['pred_No', 'pred_Yes'], columns = ['act_No', 'act_Yes'])
con_mat

,act_No,act_Yes
pred_No,168,135
pred_Yes,118,191


In [65]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    x = pd.get_dummies(data = df, columns = cat_var) 
    promotion = lgr.predict(x)
    
    my_map = {0: "No", 1: "Yes"}
    promotion = np.array([my_map[i] for i in promotion])
    
    return promotion

    

In [66]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0179.

Your nir with this strategy is 249.85.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.017867227786427138, 249.85000000000014)